In [1]:
import os 
import pandas as pd
import re

In [5]:
work_dir = '/g100_work/IscrC_mental'
wdata_dir = os.path.join(work_dir, 'data')
uc_dir = os.path.join(os.path.join(wdata_dir, 'user_classification'),'german_data')

In [106]:
df = pd.read_pickle(os.path.join(uc_dir,"german_users_with_birthyear.pkl"))

In [107]:
df_tweets = pd.read_pickle(os.path.join(uc_dir,"german_tweets.pkl"))

In [108]:
needed_format = pd.read_pickle(os.path.join(os.path.join(wdata_dir, 'user_classification/data_for_models_test.pkl')))

In [109]:
#restrict to users for which there is a gender and birth year
df.loc[df['male']==1,'is_male'] = 1
df.loc[df['female']==1,'is_male'] = 0
clean_df = df.loc[(df['is_male'].notna()) & (df['birthyear'].notna())].copy()
#manual correction
clean_df.loc[clean_df['user_id']==803169463197691904,'is_male']=1
clean_df = clean_df[['user_id','is_male','birthyear','bio']]

In [110]:
#select last datea of tweeting
df_tweets['date']=pd.to_datetime(df_tweets.created_at)
max_dates = df_tweets.groupby('user_id').agg({'date':max}).reset_index()
max_dates['last_year'] = max_dates['date'].dt.year

In [111]:
clean_df = clean_df.merge(max_dates[['user_id','last_year']], on='user_id',how='inner',validate='1:1')

In [112]:
df_all = pd.DataFrame()
for regex in ["geb[\. ].*?([12][09][0-9][0-9])",
              "geboren.+?([12][09][0-9][0-9])",
              "born.+?([12][09][0-9][0-9])",
              "birthday.+?([12][09][0-9][0-9])",
              "bday.+?([12][09][0-9][0-9])",
              "([12][09][0-9][0-9]).*?geb[\. ]",
              "([12][09][0-9][0-9]).*?geboren",
              "\*.*?([12][09][0-9][0-9])",
              "([1-9][0-9]) jahre alt"]:
    
    d = clean_df.bio.str.extract(regex)
    df = clean_df.assign(age = d.astype("Int64"))
    if regex != "([1-9][0-9]) jahre alt":
        df.age = df.last_year-df.age
    
    df_all= pd.concat([df_all,df.loc[df.age.notnull()]])

df_all = df_all.drop_duplicates("user_id")

In [117]:
df_all['birthyear'] = df_all.last_year-df_all.age

In [121]:
def mask_bio(text):
    for regex in ["geb[\. ].*?([12][09][0-9][0-9])",
              "geboren.+?([12][09][0-9][0-9])",
              "born.+?([12][09][0-9][0-9])",
              "birthday.+?([12][09][0-9][0-9])",
              "bday.+?([12][09][0-9][0-9])",
              "([12][09][0-9][0-9]).*?geb[\. ]",
              "([12][09][0-9][0-9]).*?geboren",
              "\*.*?([12][09][0-9][0-9])",
              "([1-9][0-9]) jahre alt"]:
        text = re.sub(regex,'',text)
    return text

In [127]:
df_all['masked_bio'] = df_all.bio.apply(mask_bio)

In [136]:
df = df_all.merge(df_tweets[['text','date','user_id']],on='user_id',how='inner',validate='1:m')

In [140]:
df['RT'] = df['text'].str.startswith('RT @')

In [143]:
df = df[df['RT']==False].copy()

In [144]:
def transform_df(df, N=100):
    # df bio
    df_bio = df[['user_id', 'masked_bio', 'age', 'is_male']].fillna('').drop_duplicates()
    
    # process tweets
    df_text = df[['user_id', 'text', 'date', 'age', 'is_male']]
    
    # keep the N most recent text_masked
    df_text['rank'] = df_text.groupby('user_id')['date'].rank(method='first', ascending=False)
    
    # Filter out entries with rank greater than N
    df_text = df_text[df_text['rank'] <= N]
    
    df_text_grouped = df_text.groupby('user_id')['text'].agg(lambda x: '\n'.join(x)).reset_index()
    df_text_grouped = df_text_grouped.rename(columns={'text': 'long_text'})
    
    # Merge df_bio with df_text_grouped
    result_df = pd.merge(df_bio, df_text_grouped, on='user_id', how='inner')

    return result_df

dft = transform_df(df, N=100)

/scratch_local/slurm_job.10923155/ipykernel_3248432/2838337157.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['rank'] = df_text.groupby('user_id')['date'].rank(method='first', ascending=False)


In [149]:
dft.to_pickle(os.path.join(uc_dir,'data_for_models_german_data.pkl'))